In [1]:
import time as tm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

print(f"Pandas version: {pd.__version__}")

Pandas version: 1.3.0


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [3]:
df = pd.read_csv('./data/Moscow_districts(old_borders).csv')
df

,Name,Name_rus,Population
0,Aeroport (Аэропорт)74775,NaN,NaN
1,Begovoy (Беговой)44385,NaN,NaN
2,Beskudnikovsky (Бескудниковский)74790,NaN,NaN
3,Dmitrovsky (Дмитровский)88931,NaN,NaN
4,Golovinsky (Головинский)102160,NaN,NaN
...,...,...,...
113,Pokrovskoye-Streshnevo (Покровское-Стрешнево)4...,NaN,NaN
114,Severnoe Tushino (Северное Тушино)138533,NaN,NaN
115,Shchukino (Щукино)89454,NaN,NaN
116,Strogino (Строгино)124149,NaN,NaN


#### Add population

In [4]:
df['Population'] = df['Name'].str.extract(r'(\d+)', expand=False).astype(int)
df['Name_rus']   = df['Name'].str.extract(r'([А-Яа-яЁё\-\s]+[А-Яа-яЁё])\)', expand=False)
df['Name'] = df.apply(lambda x : re.search(r'([a-zA-Z\-\s]+[a-zA-Z])', x['Name']).group(0), axis=1)
df

,Name,Name_rus,Population
0,Aeroport,Аэропорт,74775
1,Begovoy,Беговой,44385
2,Beskudnikovsky,Бескудниковский,74790
3,Dmitrovsky,Дмитровский,88931
4,Golovinsky,Головинский,102160
...,...,...,...
113,Pokrovskoye-Streshnevo,Покровское-Стрешнево,46707
114,Severnoe Tushino,Северное Тушино,138533
115,Shchukino,Щукино,89454
116,Strogino,Строгино,124149


#### Add borders of district

In [5]:
with open('./data/mo.geojson', "r") as outfile:
    moscow_geo = json.load(outfile)

In [6]:
print(moscow_geo['features'][1])

{'type': 'Feature', 'properties': {'NAME': 'Филёвский Парк', 'OKATO': '45268595', 'OKTMO': '45328000', 'NAME_AO': 'Западный', 'OKATO_AO': '45268000', 'ABBREV_AO': 'ЗАО', 'TYPE_MO': 'Муниципальный округ'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[37.42765, 55.74821], [37.42849, 55.74875], [37.42958, 55.74939], [37.43058, 55.74976], [37.43162, 55.75005], [37.4318, 55.75008], [37.43206, 55.75012], [37.43224, 55.75014], [37.43265, 55.75019], [37.43296, 55.75012], [37.43513, 55.74964], [37.43698, 55.74925], [37.43661, 55.74872], [37.43635, 55.74787], [37.43669, 55.7471], [37.43738, 55.74642], [37.43974, 55.74522], [37.44248, 55.74357], [37.44446, 55.74261], [37.44652, 55.74186], [37.44927, 55.74113], [37.45253, 55.7406], [37.45532, 55.7406], [37.45914, 55.74121], [37.4618, 55.74186], [37.4648, 55.7429], [37.46738, 55.74403], [37.47102, 55.74611], [37.47278, 55.74727], [37.47416, 55.7485], [37.47514, 55.74988], [37.47579, 55.75123], [37.47617, 55.75229], [37.47652, 55.75459], [37.47

In [7]:
dic = {'Name_rus': [], 'Coordinates': []}

In [8]:
for district in moscow_geo['features']:
    dic['Name_rus'].append(district['properties']['NAME'])
    dic['Coordinates'].append(district['geometry']['coordinates'][0])

In [9]:
df_coordinates = pd.DataFrame(dic, columns=['Name_rus', 'Coordinates'])

In [10]:
df = pd.merge(df, df_coordinates, how='left')

In [11]:
df

,Name,Name_rus,Population,Coordinates
0,Aeroport,Аэропорт,74775,"[[37.51101, 55.80608], [37.51137, 55.80647], [..."
1,Begovoy,Беговой,44385,"[[37.54084, 55.78496], [37.54188, 55.78541], [..."
2,Beskudnikovsky,Бескудниковский,74790,"[[37.52966, 55.87283], [37.52982, 55.87291], [..."
3,Dmitrovsky,Дмитровский,88931,"[[37.49625, 55.89248], [37.49851, 55.89347], [..."
4,Golovinsky,Головинский,102160,"[[37.46446, 55.84712], [37.46569, 55.84756], [..."
...,...,...,...,...
113,Pokrovskoye-Streshnevo,Покровское-Стрешнево,46707,"[[37.39444, 55.8291], [37.395, 55.83041], [37...."
114,Severnoe Tushino,Северное Тушино,138533,"[[37.39374, 55.85179], [37.39505, 55.85432], [..."
115,Shchukino,Щукино,89454,"[[37.4461, 55.79449], [37.44618, 55.79531], [3..."
116,Strogino,Строгино,124149,"[[37.36726, 55.79036], [37.36784, 55.79136], [..."


In [12]:
# fix list in list
df['Coordinates'] = df['Coordinates'].apply(lambda x: x[0] if isinstance(x[0][0], list) else x)

Is in polygon

In [13]:
def isInPolygon(point, polygon) -> bool:
    isInside = False
    minLat = polygon[0][0]
    minLon = polygon[0][0]
    maxLat = polygon[0][1]
    maxLon = polygon[0][1]
    
    for poly in polygon:
        minLat = min(poly[0], minLat)
        maxLat = max(poly[0], maxLat)
        minLon = min(poly[1], minLon)
        maxLon = max(poly[1], maxLon)

    if (point[0] < minLat or point[0] > maxLat or point[1] < minLon or point[1] > maxLon):
        return False
    
    j = len(polygon) - 1
    for i in range(len(polygon)):
        if ( (polygon[i][1] > point[1]) != (polygon[j][1] > point[1]) and
            point[0] < (polygon[j][0] - polygon[i][0]) * (point[1] - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0] ):
            isInside = not isInside
        j = i
    
    return isInside;

In [14]:
print(isInPolygon([37.524297, 55.804467], df.iloc[0]['Coordinates']))
print(isInPolygon([37.534755, 55.805586], df.iloc[0]['Coordinates']))
print(isInPolygon([39.534755, 55.805586], df.iloc[0]['Coordinates']))
print(isInPolygon([19., 75.], df.iloc[0]['Coordinates']))

True
True
False
False


#### Add EV stations

In [15]:
df_stations = pd.read_excel('./data/ЭЗС.xlsx')

In [16]:
df_stations

,lat,lon,name,popularity,rubric
0,55.643995,37.663501,ЭЗС быстрой зарядки МГТС,4,Станция зарядки электромобилей
1,55.609760,37.538185,ЭЗС медленной зарядки МГТС,4,Станция зарядки электромобилей
2,55.778786,37.719588,Станция зарядки электромобилей,4,Станция зарядки электромобилей
3,55.656473,37.536152,МОЭСК-EV,4,Станция зарядки электромобилей
4,55.864185,37.673748,Станция зарядки электромобилей,4,Станция зарядки электромобилей
...,...,...,...,...,...
145,55.759599,37.641528,Зарядная станция,0,Станция зарядки электромобилей
146,55.736547,37.687836,Энергия Москвы,0,Станция зарядки электромобилей
147,55.688365,37.355699,Зарядная станция,0,Станция зарядки электромобилей
148,55.756887,37.637893,Зарядная станция,0,Станция зарядки электромобилей


In [17]:
def calNumStations(coord):
    num = 0
    for _, station in df_stations.iterrows():
        if isInPolygon([station['lon'], station['lat']], coord):
            num += 1
    return num

In [18]:
df['Num_stations'] = df['Coordinates'].apply(calNumStations)

In [19]:
df.sort_values('Num_stations')

,Name,Name_rus,Population,Coordinates,Num_stations
0,Aeroport,Аэропорт,74775,"[[37.51101, 55.80608], [37.51137, 55.80647], [...",0
73,Chertanovo Severnoye,Чертаново Северное,104613,"[[37.57727, 55.63268], [37.57741, 55.63436], [...",0
72,Brateyevo,Братеево,94644,"[[37.72713, 55.63526], [37.72768, 55.63525], [...",0
71,Biryulyovo Zapadnoye,Бирюлёво Западное,83303,"[[37.61313, 55.57463], [37.61456, 55.58613], [...",0
70,Biryulyovo Vostochnoye,Бирюлёво Восточное,129700,"[[37.64076, 55.61051], [37.64207, 55.61115], [...",0
...,...,...,...,...,...
21,Tagansky,Таганский,109993,"[[37.63211, 55.74885], [37.63237, 55.74952], [...",8
20,Presnensky,Пресненский,116979,"[[37.51393, 55.75842], [37.51567, 55.75909], [...",9
24,Zamoskvorechye,Замоскворечье,5059,"[[37.61513, 55.72688], [37.61761, 55.72708], [...",9
16,Basmanny,Басманный,100899,"[[37.6279, 55.75954], [37.62807, 55.75961], [3...",9


#### Add cost per meter of apartments

In [20]:
cost_of_districts = []
with open('./data/costs.txt', encoding="utf8") as fp:
    print("Name of the file: ", fp.name)
    line = fp.readline()
    while line:
        line_to_list = line.split(r' - ')
        districts = line_to_list[0].split(r', ')
        for d in districts:
            lst = []
            lst.append(d)
            lst.append(int(line_to_list[-1].split(r' ')[0])*1000 + int(line_to_list[-1].split(r' ')[1]))
            cost_of_districts.append(lst)
            print(d)
            print(int(line_to_list[-1].split(r' ')[0])*1000 + int(line_to_list[-1].split(r' ')[1]))
        line = fp.readline()

Name of the file:  ./data/costs.txt
Китай-город
380000
Остоженка
373000
Тверской
356700
Якиманка
331586
Арбат
331000
Хамовники
309700
Пресненский
291518
Донской
280000
Красносельский
278100
Замоскворечье
277000
Мещанский
270500
Таганский
270000
Дорогомилово
258100
Беговой
231700
Раменки
221700
Гагаринский
221700
Ломоносовский
221700
Проспект Вернадского
214800
Басманный
207500
Сокол
207200
Аэропорт
207200
Черёмушки
201277
Сокольники
199000
Хорошёвский
197400
Академический
197000
Савёловский
195000
Марьина Роща
194700
Алексеевский
194100
Хорошёво-Мнёвники
189900
Крылатское
189200
Обручевский
189100
Коньково
189000
Филёвский Парк
188900
Преображенское
187400
Нижегородский
185000
Тропарёво-Никулино
180400
Ростокино
179900
Останкинский
179900
Тимирязевский
172700
Бутырский
172500
Войковский
172200
Коптево
172200
Соколиная Гора
171400
Южнопортовый
170000
Щукино
169900
Покровское-Стрешнево
169900
Даниловский
169000
Котловка
169000
Фили-Давыдково
168000
Можайский
168000
Нагорный
167900
Зюзино

In [21]:
temp = pd.DataFrame(cost_of_districts, columns=['Name_rus', 'Cost per meter'])
#temp = temp.sort_values(by=['Name_rus']).reset_index().drop('index', axis=1)
df = df.merge(temp, on='Name_rus', how='left')
df

,Name,Name_rus,Population,Coordinates,Num_stations,Cost per meter
0,Aeroport,Аэропорт,74775,"[[37.51101, 55.80608], [37.51137, 55.80647], [...",0,207200
1,Begovoy,Беговой,44385,"[[37.54084, 55.78496], [37.54188, 55.78541], [...",0,231700
2,Beskudnikovsky,Бескудниковский,74790,"[[37.52966, 55.87283], [37.52982, 55.87291], [...",1,132700
3,Dmitrovsky,Дмитровский,88931,"[[37.49625, 55.89248], [37.49851, 55.89347], [...",0,132700
4,Golovinsky,Головинский,102160,"[[37.46446, 55.84712], [37.46569, 55.84756], [...",1,154400
...,...,...,...,...,...,...
113,Pokrovskoye-Streshnevo,Покровское-Стрешнево,46707,"[[37.39444, 55.8291], [37.395, 55.83041], [37....",1,169900
114,Severnoe Tushino,Северное Тушино,138533,"[[37.39374, 55.85179], [37.39505, 55.85432], [...",0,155200
115,Shchukino,Щукино,89454,"[[37.4461, 55.79449], [37.44618, 55.79531], [3...",0,169900
116,Strogino,Строгино,124149,"[[37.36726, 55.79036], [37.36784, 55.79136], [...",0,167400


#### Add free power

In [22]:
df['Free_power'] = 0

In [41]:
df.loc[df['Name_rus'] == 'Южное Тушино', 'Free_power'] = 92.28 / 2
df.loc[df['Name_rus'] == 'Северное Тушино','Free_power'] = 92.28 / 2
df.loc[df['Name_rus'] == 'Ховрино', 'Free_power'] = 42.93 / 2
df.loc[df['Name_rus'] == 'Левобережный', 'Free_power'] = 42.93 / 2
df.loc[df['Name_rus'] == 'Строгино', 'Free_power'] = 36.87 / 2
df.loc[df['Name_rus'] == 'Митино', 'Free_power'] = 36.87
df.loc[df['Name_rus'] == 'Покровское-Стрешнево', 'Free_power'] = 36.87 / 2
df.loc[df['Name_rus'] == 'Щукино', 'Free_power'] = 29.5
df.loc[df['Name_rus'] == 'Хорошёво-Мнёвники', 'Free_power'] = 31.09
df.loc[df['Name_rus'] == 'Крылатское', 'Free_power'] = 43.07
df.loc[df['Name_rus'] == 'Кунцево', 'Free_power'] = 162.93 / 3
df.loc[df['Name_rus'] == 'Можайский', 'Free_power'] = 162.93 / 3
df.loc[df['Name_rus'] == 'Очаково-Матвеевское', 'Free_power'] = 162.93 / 3
df.loc[df['Name_rus'] == 'Фили-Давыдково', 'Free_power'] = 9.95
df.loc[df['Name_rus'] == 'Филёвский Парк', 'Free_power'] = 156.78 / 2
df.loc[df['Name_rus'] == 'Дорогомилово', 'Free_power'] = 156.78 / 2
df.loc[df['Name_rus'] == 'Ново-Переделкино', 'Free_power'] = 53.43
df.loc[df['Name_rus'] == 'Солнево', 'Free_power'] = 165.25
df.loc[df['Name_rus'] == 'Тропарёво-Никулино', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Проспект Вернадского', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Ломоносовский', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Академический', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Гагаринский', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Раменки', 'Free_power'] = 309.14 / 6
df.loc[df['Name_rus'] == 'Обручевский', 'Free_power'] = 40 / 2
df.loc[df['Name_rus'] == 'Черёмушки', 'Free_power'] = 40 / 2
df.loc[df['Name_rus'] == 'Тёплый Стан', 'Free_power'] = 48 / 2
df.loc[df['Name_rus'] == 'Коньково', 'Free_power'] = 48 / 2
df.loc[df['Name_rus'] == 'Котловка', 'Free_power'] = 22 / 2
df.loc[df['Name_rus'] == 'Нагорный', 'Free_power'] = 22 / 2
df.loc[df['Name_rus'] == 'Ясенево', 'Free_power'] = 58 / 2
df.loc[df['Name_rus'] == 'Зюзино', 'Free_power'] = 58 / 2
df.loc[df['Name_rus'] == 'Чертаново Центральное', 'Free_power'] = 132.12 / 3
df.loc[df['Name_rus'] == 'Чертаново Южное', 'Free_power'] = 132.12 / 3
df.loc[df['Name_rus'] == 'Чертаново Северное', 'Free_power'] = 132.12 / 3
df.loc[df['Name_rus'] == 'Северное Бутово', 'Free_power'] = 275 / 3
df.loc[df['Name_rus'] == 'Южное Бутово', 'Free_power'] = 275 / 3
df.loc[df['Name_rus'] == 'Бирюлёво Западное', 'Free_power'] = 79.66 / 2
df.loc[df['Name_rus'] == 'Бирюлёво Восточное', 'Free_power'] = 79.66 / 2
df.loc[df['Name_rus'] == 'Царицыно', 'Free_power'] = 129.89 / 2
df.loc[df['Name_rus'] == 'Москворечье-Сабурово', 'Free_power'] = 129.89 / 2
df.loc[df['Name_rus'] == 'Орехово-Борисово Северное', 'Free_power'] = 11
df.loc[df['Name_rus'] == 'Орехово-Борисово Южное', 'Free_power'] = 30.69
df.loc[df['Name_rus'] == 'Братеево', 'Free_power'] = 53 / 2
df.loc[df['Name_rus'] == 'Зябликово', 'Free_power'] = 53 / 2
df.loc[df['Name_rus'] == 'Люблино', 'Free_power'] = 268.45 / 5
df.loc[df['Name_rus'] == 'Капотня', 'Free_power'] = 268.45 / 5
df.loc[df['Name_rus'] == 'Марьино', 'Free_power'] = 268.45 / 5
df.loc[df['Name_rus'] == 'Кузьминки', 'Free_power'] = 268.45 / 5
df.loc[df['Name_rus'] == 'Текстильщики', 'Free_power'] = 268.45 / 5
df.loc[df['Name_rus'] == 'Нагатино-Садовники', 'Free_power'] = 93.56 / 2
df.loc[df['Name_rus'] == 'Нагатинский Затон', 'Free_power'] = 93.56 / 2
df.loc[df['Name_rus'] == 'Печатники', 'Free_power'] = 69.29
df.loc[df['Name_rus'] == 'Выхино-Жулебино', 'Free_power'] = 107
df.loc[df['Name_rus'] == 'Рязанский', 'Free_power'] = 33.30
df.loc[df['Name_rus'] == 'Вешняки', 'Free_power'] = 57.09 / 3
df.loc[df['Name_rus'] == 'Новогиреево', 'Free_power'] = 57.09 / 3
df.loc[df['Name_rus'] == 'Ивановское', 'Free_power'] = 57.09 / 3
df.loc[df['Name_rus'] == 'Перово', 'Free_power'] = 93.96 / 3
df.loc[df['Name_rus'] == 'Измайлово', 'Free_power'] = 93.96 / 3
df.loc[df['Name_rus'] == 'Восточное Измайлово', 'Free_power'] = 93.96 / 3
df.loc[df['Name_rus'] == 'Нижегородский', 'Free_power'] = 68.63
df.loc[df['Name_rus'] == 'Южнопортовый', 'Free_power'] = 189.82
df.loc[df['Name_rus'] == 'Лефортово', 'Free_power'] = 38.47
df.loc[df['Name_rus'] == 'Новокосино', 'Free_power'] = 68.63
df.loc[df['Name_rus'] == 'Восточный', 'Free_power'] = 68.63
df.loc[df['Name_rus'] == 'Соколиная Гора', 'Free_power'] = 219.50 / 4
df.loc[df['Name_rus'] == 'Преображенское', 'Free_power'] = 219.50 / 4
df.loc[df['Name_rus'] == 'Сокольники', 'Free_power'] = 219.50 / 4
df.loc[df['Name_rus'] == 'Богородское', 'Free_power'] = 219.50 / 4
df.loc[df['Name_rus'] == 'Метрогородок', 'Free_power'] = 109.7 / 3
df.loc[df['Name_rus'] == 'Гольяново', 'Free_power'] = 109.7 / 3
df.loc[df['Name_rus'] == 'Северное Измайлово', 'Free_power'] = 109.7 / 3
df.loc[df['Name_rus'] == 'Ярославский', 'Free_power'] = 35.47
df.loc[df['Name_rus'] == 'Бабушкинский', 'Free_power'] = 68.86 / 5
df.loc[df['Name_rus'] == 'Южное Медведково', 'Free_power'] = 68.86 / 5
df.loc[df['Name_rus'] == 'Северное Медведково', 'Free_power'] = 68.86 / 5
df.loc[df['Name_rus'] == 'Бибирево', 'Free_power'] = 68.86 / 5
df.loc[df['Name_rus'] == 'Лосиноостровский', 'Free_power'] = 68.86 / 5
df.loc[df['Name_rus'] == 'Свиблово', 'Free_power'] = 66 / 2
df.loc[df['Name_rus'] == 'Ростокино', 'Free_power'] = 66 / 2
df.loc[df['Name_rus'] == 'Отрадное', 'Free_power'] = 21.58
df.loc[df['Name_rus'] == 'Алтуфьевский', 'Free_power'] = 41.45 / 2
df.loc[df['Name_rus'] == 'Лианозово', 'Free_power'] = 41.45 / 2
df.loc[df['Name_rus'] == 'Головинский', 'Free_power'] = 188.58 / 5
df.loc[df['Name_rus'] == 'Дмитровский', 'Free_power'] = 188.58 / 5
df.loc[df['Name_rus'] == 'Западное Дегунино', 'Free_power'] = 188.58 / 5
df.loc[df['Name_rus'] == 'Восточное Дегунино', 'Free_power'] = 188.58 / 5
df.loc[df['Name_rus'] == 'Бескудниковский', 'Free_power'] = 188.58 / 5
df.loc[df['Name_rus'] == 'Коптево', 'Free_power'] = 109.47 / 4
df.loc[df['Name_rus'] == 'Сокол', 'Free_power'] = 109.47 / 4
df.loc[df['Name_rus'] == 'Войковский', 'Free_power'] = 109.47 / 4
df.loc[df['Name_rus'] == 'Тимирязевский', 'Free_power'] = 109.47 / 4
df.loc[df['Name_rus'] == 'Марьина Роща', 'Free_power'] = 170.41 / 5
df.loc[df['Name_rus'] == 'Марфино', 'Free_power'] = 170.41 / 5
df.loc[df['Name_rus'] == 'Бутырский', 'Free_power'] = 170.41 / 5
df.loc[df['Name_rus'] == 'Останкинский', 'Free_power'] = 170.41 / 5
df.loc[df['Name_rus'] == 'Алексеевский', 'Free_power'] = 170.41 / 5
df.loc[df['Name_rus'] == 'Хорошёвский', 'Free_power'] = 251 / 3
df.loc[df['Name_rus'] == 'Савёловский', 'Free_power'] = 251 / 3
df.loc[df['Name_rus'] == 'Аэропорт', 'Free_power'] = 251 / 3
df.loc[df['Name_rus'] == 'Беговой', 'Free_power'] = 18
df.loc[df['Name_rus'] == 'Пресненский', 'Free_power'] = 260.54
df.loc[df['Name_rus'] == 'Тверской', 'Free_power'] = 82
df.loc[df['Name_rus'] == 'Мещанский', 'Free_power'] = 78.77
df.loc[df['Name_rus'] == 'Красносельский', 'Free_power'] = 103.15
df.loc[df['Name_rus'] == 'Басманный', 'Free_power'] = 127.83
df.loc[df['Name_rus'] == 'Таганский', 'Free_power'] = 14.46 / 2
df.loc[df['Name_rus'] == 'Замоскворечье', 'Free_power'] = 14.46 / 2
df.loc[df['Name_rus'] == 'Якиманка', 'Free_power'] = 132.40 / 3
df.loc[df['Name_rus'] == 'Хамовники', 'Free_power'] = 132.40 / 3
df.loc[df['Name_rus'] == 'Арбат', 'Free_power'] = 132.40 / 3
df.loc[df['Name_rus'] == 'Куркино', 'Free_power'] = 5
df.loc[df['Name_rus'] == 'Даниловский', 'Free_power'] = 515.8 / 2
df.loc[df['Name_rus'] == 'Донской', 'Free_power'] = 515.8 / 2
df.loc[df['Name_rus'] == 'Солнцево', 'Free_power'] = 165.25
df.loc[df['Name_rus'] == 'Некрасовка', 'Free_power'] = 25
df.loc[df['Name_rus'] == 'Косино-Ухтомский', 'Free_power'] = 39
df.loc[df['Name_rus'] == 'Северный', 'Free_power'] = 32

In [42]:
df.sort_values('Free_power')

,Name,Name_rus,Population,Coordinates,Num_stations,Cost per meter,Free_power,Power_per_person
111,Kurkino,Куркино,2339,"[[37.36177, 55.90146], [37.36179, 55.90155], [...",4,147800,5.000000,0.008551
24,Zamoskvorechye,Замоскворечье,5059,"[[37.61513, 55.72688], [37.61761, 55.72708], [...",9,277000,7.230000,0.001429
21,Tagansky,Таганский,109993,"[[37.63211, 55.74885], [37.63237, 55.74952], [...",8,270000,7.230000,0.000066
100,Fili-Davydkovo,Фили-Давыдково,92965,"[[37.43797, 55.73717], [37.43895, 55.73689], [...",0,168000,9.950000,0.000107
82,Orekhovo-Borisovo Severnoye,Орехово-Борисово Северное,121402,"[[37.67127, 55.62009], [37.67336, 55.62151], [...",0,140800,11.000000,0.000091
...,...,...,...,...,...,...,...,...
108,Solntsevo,Солнцево,85642,"[[37.36785, 55.6532], [37.36977, 55.65952], [3...",0,134300,165.250000,0.001930
69,Yuzhnoportovy,Южнопортовый,60056,"[[37.66069, 55.7307], [37.66082, 55.73073], [3...",0,170000,189.820000,0.003161
76,Danilovsky,Даниловский,90265,"[[37.60622, 55.71094], [37.60627, 55.71115], [...",8,169000,257.900000,0.002857
77,Donskoy,Донской,45477,"[[37.58213, 55.711], [37.58277, 55.71125], [37...",3,280000,257.900000,0.005671


In [43]:
df['Power_per_person'] = df['Free_power'] / df['Population']

In [44]:
df.sort_values('Power_per_person')

,Name,Name_rus,Population,Coordinates,Num_stations,Cost per meter,Free_power,Power_per_person
21,Tagansky,Таганский,109993,"[[37.63211, 55.74885], [37.63237, 55.74952], [...",8,270000,7.230000,0.000066
82,Orekhovo-Borisovo Severnoye,Орехово-Борисово Северное,121402,"[[37.67127, 55.62009], [37.67336, 55.62151], [...",0,140800,11.000000,0.000091
28,Bibirevo,Бибирево,151334,"[[37.58668, 55.88721], [37.5868, 55.89123], [3...",1,137100,13.772000,0.000091
100,Fili-Davydkovo,Фили-Давыдково,92965,"[[37.43797, 55.73717], [37.43895, 55.73689], [...",0,168000,9.950000,0.000107
37,Severnoye Medvedkovo,Северное Медведково,111804,"[[37.61965, 55.88561], [37.62001, 55.88564], [...",0,145400,13.772000,0.000123
...,...,...,...,...,...,...,...,...
63,Nekrasovka,Некрасовка,7803,"[[37.90613, 55.70626], [37.90701, 55.70699], [...",0,129800,25.000000,0.003204
38,Severny,Северный,9629,"[[37.51948, 55.94176], [37.52004, 55.94185], [...",0,132500,32.000000,0.003323
80,Nagatinsky zaton,Нагатинский Затон,10594,"[[37.65342, 55.65539], [37.65343, 55.65601], [...",1,160100,46.780000,0.004416
57,Vostochny,Восточный,12700,"[[37.84533, 55.81356], [37.85288, 55.81672], [...",0,129900,68.630000,0.005404


In [47]:
df[['Name_rus', 'Free_power']].to_csv('./data/freepower.csv')